In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from custom_mlp import CustomMLP_4_8_1



app = Flask(__name__)

CORS(app,resources={"r/*":{"origins":"*"}})

@app.after_request
def handle_cors(response):
    response.headers["Access-Control-Allow-Headers"]="Content-Type,Authorization"
    response.headers["Access-Control-Allow-Methods"]="GET,POST,OPTIONS"
    response.headers['Access-Control-Allow-Origin']="*"
    return response

NB_MODEL_PATH = "naive_bayes_model.pkl"
MLP_MODEL_PATH = "mlp_model.pkl"

# Load models
try:
    with open(NB_MODEL_PATH, 'rb') as f:
        loaded_nb_model = pickle.load(f)
    print(f"Loaded Naive Bayes model from {NB_MODEL_PATH}")
except FileNotFoundError:
    print(f"Warning: Naive Bayes model not found at {NB_MODEL_PATH}")
    loaded_nb_model = None


try:
    with open(MLP_MODEL_PATH, 'rb') as f:
        loaded_mlp_model = pickle.load(f)
    print(f"Loaded MLP model from {MLP_MODEL_PATH}")
except FileNotFoundError:
    print(f"Warning: MLP model not found at {MLP_MODEL_PATH}")
    loaded_mlp_model = None


@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()

      
        model_type = data.get('model_type', 'naive_bayes').lower()

        
        age = float(data['age'])
        glucose = float(data['glucose'])
        insulin = float(data['insulin'])
        bmi = float(data['bmi'])

        input_features = np.array([[age, glucose, insulin, bmi]])

        
        if model_type == 'naive_bayes':
            if loaded_nb_model is None:
                return jsonify({'error': 'Naive Bayes model not loaded'}), 500

            prediction = loaded_nb_model.predict(input_features)
            
            probabilities = loaded_nb_model.predict_proba(input_features)
            confidence = float(np.max(probabilities))

            return jsonify({
                'diabetes_type': int(prediction[0]),
                'model_used': 'naive_bayes',
                'confidence': confidence
            }),201

        elif model_type == 'mlp':
            if loaded_mlp_model is None:
                return jsonify({'error': 'MLP model not loaded'}), 500
            
            prediction = loaded_mlp_model.predict(input_features)

            return jsonify({
                'diabetes_type': int(prediction[0]),
                'model_used': 'mlp',
                'confidence': None  
            }),201

        else:
            return jsonify({'error': f'Invalid model type: {model_type}. Use "naive_bayes" or "mlp"'}), 400

    except KeyError as e:
        return jsonify({'error': f'Missing required field: {str(e)}'}), 400
    except ValueError as e:
        return jsonify({'error': f'Invalid input values: {str(e)}'}), 400
    except Exception as e:
        return jsonify({'error': f'Prediction error: {str(e)}'}), 500


@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'ok',
        'naive_bayes_loaded': loaded_nb_model is not None,
        'mlp_loaded': loaded_mlp_model is not None
    }),200


@app.route('/', methods=['GET'])
def home():
    return jsonify({
        'message': 'Diabetes Prediction API',
        'endpoints': {
            'POST /predict': 'Make a prediction',
            'GET /health': 'Health check'
        },
        'example_request': {
            'age': 45,
            'glucose': 120,
            'insulin': 45,
            'bmi': 28.5,
            'model_type': 'naive_bayes'
        }
    }),200


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)
